In [ ]:
# stream tweets from Twitter using the keyword "BTS", for 30 minute increments over 4 days
import tweepy
import time
from tweepy import OAuthHandler
from tweepy import API
from tweepy.streaming import StreamListener
from tweepy import Stream
import datetime

C_KEY = 'TR4gOd7J3cA5qCpWAMPj9Y9ls'
C_SECRET = 'QbqygSH6yR5heLvXhgZSomZPDRhJUowYEWfskj14G3lJ03EFJv'
A_TOKEN_KEY = '908444306867998720-p0yN6CSH2oM5xmVX2qxrW12vX03VLBP'
A_TOKEN_SECRET = 'ZOv9Doo3DjC4oOqnGSONlEFX7DaG3AseMbveCUO3jn2Tw'

# create a StreamListener class ADAPTED from lecture notes
class MyListener(StreamListener):
    def __init__(self, time_limit=10):
        self.start_time = time.time()
        self.limit = time_limit
        now = datetime.datetime.now()
        outname = "bts-" + now.strftime("%m-%d-%Y") + ".json"
        self.outFile = open('bts.json', 'w')
        super(MyListener, self).__init__()
        
    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.outFile.write(data)
            return True
        else:
            self.outFile.close()
            return False
        
    def on_error(self, status):
        print(status)

auth = tweepy.OAuthHandler(C_KEY, C_SECRET)
auth.set_access_token(A_TOKEN_KEY, A_TOKEN_SECRET)
api = tweepy.API(auth)

myStream = Stream(auth, MyListener(time_limit=30))
myStream.filter(track=['BTS'])

In [59]:
# names of the json files that represents the daily tweets
json_date = '11-06-2017/'
bts_json = 'bts-11-06-2017.json'

In [60]:
# used from http://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [61]:
# dictionary containing the inverted states 
# i.e. the format is "'Michigan': 'MI'"
inverted_states = dict((v,k) for k, v in states.iteritems())

In [62]:
# dict containing tweets per state
# initialize to 0 first
state_twts = dict((k, []) for k, v in states.iteritems()) 

In [63]:
# load the streamed tweets
import json
data = []
with open(bts_json) as f:
    for line in f:
        data.append(json.loads(line))
#len(data)

6952

In [64]:
# filter tweets that are retweeted and are missing location
num_twts = len(data)
filtered_twts = []
for i in range(0, num_twts):
    try:
        if not data[i]['retweeted'] and 'RT @'not in data[i]['text'] and data[i]['user']['location']:
            #print(data[i]['text'])
            filtered_twts.append(data[i])
    except KeyError:
        pass
    except ValueError:
        pass
    except TypeError:
        # if tweet doesn't have place defined
        pass
        
#print("number of tweets: {}", len(filtered_twts))

('number of tweets: {}', 488)


In [65]:
# checking for tweets in the united states
us = ("USA", "US", "United States")

# get location
size_ = len(filtered_twts)
for i in range(0, size_):
    location = filtered_twts[i]['user']['location']
    if ',' in location:
        location_list = [txt.strip() for txt in location.split(',')]
        
        sz_ = len(location_list)
        
        for j in range(0, sz_):
            if location_list[j] in states:
                state_twts[location_list[j]].append(filtered_twts[i])
            elif location_list[j] in us:
                # the format may be: 'Texas, USA' or 'California, US'
                cityorstate = location_list[j-1]  
                if cityorstate in inverted_states:
                    state_twts[inverted_states[cityorstate]].append(filtered_twts[i])



In [66]:
# check tweets per state and save it to it's own file under the folder of its date
import json

for k,v in sorted(state_twts.items(), key=lambda kv: kv[1], reverse=True):
    print("{} : {} ".format(k, v))
    
    output_name = json_date + k + ".json"
    with open(output_name, 'w') as fout:
        json.dump(v, fout)
    
    fout.close()
    
    

MI : [{u'quote_count': 0, u'contributors': None, u'truncated': False, u'text': u'Lmfaooooo \U0001f62d\U0001f62d\U0001f62d https://t.co/eopLYVX9Y2', u'is_quote_status': True, u'in_reply_to_status_id': None, u'reply_count': 0, u'id': 927741475378606080, u'favorite_count': 0, u'entities': {u'user_mentions': [], u'symbols': [], u'hashtags': [], u'urls': [{u'url': u'https://t.co/eopLYVX9Y2', u'indices': [14, 37], u'expanded_url': u'https://twitter.com/taetrbi/status/927311991563309056', u'display_url': u'twitter.com/taetrbi/status\u2026'}]}, u'quoted_status_id': 927311991563309056, u'retweeted': False, u'coordinates': None, u'timestamp_ms': u'1510025778990', u'quoted_status': {u'quote_count': 103, u'contributors': None, u'truncated': False, u'text': u'me when bts perform fire for the millionth time https://t.co/7mJhMmbpX3', u'is_quote_status': False, u'in_reply_to_status_id': None, u'reply_count': 18, u'id': 927311991563309056, u'favorite_count': 4884, u'entities': {u'user_mentions': [], u'